In [110]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [111]:
%load_ext autoreload
%autoreload
exec(open("plot_utils.py").read())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [112]:
country_name = 'Sweden'
file_name = '../output/2020-05-14-n1-normal-1M/inference/{}.npz'.format(country_name)
data = np.load(file_name)



In [150]:
var_names = list(data['var_names'])
incidences = data['incidences']
chain = data['chain']
nsteps, ndim, N, Td1, Td2, model_type = data['params']
start_date = data['start_date']
nburn = int(nsteps * 0.6)
sample = chain[:, nburn:, :].reshape(-1, ndim)
sample[:,var_names.index('τ')] = sample[:,var_names.index('τ')].astype(int)
sample[:,var_names.index('Δt0')] = sample[:,var_names.index('Δt0')].astype(int)

medians = [np.median(sample[:,i]) for i in range(len(var_names))]
means = [sample[:,i].mean() for i in range(len(var_names))]
lnprobability = data['lnprobability'][:, nburn:]
MAP = sample[lnprobability.argmax()]

In [151]:
ModelClass = get_model_class(model_type)
model = ModelClass(country_name, incidences, pd.to_datetime(start_date), N, get_last_NPI_date(country_name), get_first_NPI_date(country_name), params_bounds, Td1, Td2)

In [152]:
list(zip(var_names, means, medians))

[('Z', 3.8710446526913485, 4.021484328906158),
 ('D', 3.4923449852657877, 3.4968874777251724),
 ('μ', 0.46504555472168363, 0.41760117575851763),
 ('β', 1.0849493162875685, 1.0593404666952613),
 ('α1', 0.18748144172666412, 0.11263327470720913),
 ('λ', 0.6413049809569022, 0.6375506446035426),
 ('α2', 0.3204803992233518, 0.25008561019046444),
 ('E0', 638.4148046056314, 386.2140431632074),
 ('Iu0', 775.9920421993572, 494.3707838336137),
 ('Δt0', 2.12733685, 2.0),
 ('τ', 16.9122945, 17.0)]

## medians

In [253]:
X = incidences
θ = medians

Z, D, μ, β, α1, λ, α2, E0, Iu0, Δt0, τ = θ
Td1 = 9
Td2 = 6

τ = int(τ) # for explanation see https://github.com/dfm/emcee/issues/150
Δt0 = int(Δt0)

total_zeros = params_bounds['Δt0'][1]
unrellevant_zeros = total_zeros - Δt0
τ = τ - unrellevant_zeros
X = X[unrellevant_zeros:]
ndays = len(X)

S, E, Ir, Iu, R, Y = model._simulate(Z, D, μ, β, α1, λ, α2, E0, Iu0, Δt0, τ, ndays)
p1 = 1/Td1
p2 = 1/Td2
Xsum = X.cumsum() 
n = Y[1:] - Xsum[:-1] 
n = np.maximum(1, n)
p = ([p1] * τ + [p2] * (ndays - τ))[1:]

loglik = scipy.stats.poisson.logpmf(X[Δt0:], n[Δt0-1:] * p[Δt0-1:])
loglik.mean()

-151.33960799159343

In [254]:
print('τ',int(τ),'Δt0',int(Δt0))

τ 14 Δt0 2


In [255]:
def r(arr):
    return [str(int(a)) if not np.isinf(a) else '-inf' for a in arr[:-1]]+[str(arr[-1])]
def r_arrays(arr):
    return [r(a) for a in arr]

loglik_to_print = np.array([0]*Δt0+list(loglik))
print('\t'.join(['day', 'loglik', 'X', 'Xsum', 'Y', 'n','p']))
arr = r_arrays(list(zip(range(len(X)),loglik_to_print,X, Xsum, Y, n, p)))
for a in arr:
    print('\t'.join(a))

day	loglik	X	Xsum	Y	n	p
0	0	0	0	0	12	0.1111111111111111
1	0	0	0	12	27	0.1111111111111111
2	-1	1	1	27	43	0.1111111111111111
3	-1	5	6	44	57	0.1111111111111111
4	-1	5	11	63	73	0.1111111111111111
5	-6	1	12	84	96	0.1111111111111111
6	-8	1	13	108	120	0.1111111111111111
7	-10	1	14	133	147	0.1111111111111111
8	-4	9	23	161	169	0.1111111111111111
9	-4	11	34	192	191	0.1111111111111111
10	-3	26	60	225	202	0.1111111111111111
11	-42	76	136	262	166	0.1111111111111111
12	-3	24	160	302	186	0.1111111111111111
13	-11	42	202	346	144	0.16666666666666666
14	-10	45	247	346	200	0.16666666666666666
15	-24	78	325	447	222	0.16666666666666666
16	-81	136	461	547	189	0.16666666666666666
17	-131	158	619	650	136	0.16666666666666666
18	-168	155	774	755	89	0.16666666666666666
19	-211	149	923	863	53	0.16666666666666666
20	-173	108	1031	976	62	0.16666666666666666
21	-115	89	1120	1093	95	0.16666666666666666
22	-21	46	1166	1215	176	0.16666666666666666
23	-101	134	1300	1342	174	0.16666666666666666
24	-85	122	1422	1474	190	0

## means

In [245]:
X = incidences
θ = means

Z, D, μ, β, α1, λ, α2, E0, Iu0, Δt0, τ = θ
Td1 = 9
Td2 = 6

τ = int(τ) # for explanation see https://github.com/dfm/emcee/issues/150
Δt0 = int(Δt0)

total_zeros = params_bounds['Δt0'][1]
unrellevant_zeros = total_zeros - Δt0
τ = τ - unrellevant_zeros
X = X[unrellevant_zeros:]
ndays = len(X)

S, E, Ir, Iu, R, Y = model._simulate(Z, D, μ, β, α1, λ, α2, E0, Iu0, Δt0, τ, ndays)
p1 = 1/Td1
p2 = 1/Td2
Xsum = X.cumsum() 
n = Y[1:] - Xsum[:-1] 
n = np.maximum(1, n)
p = ([p1] * τ + [p2] * (ndays - τ))[1:]

loglik = scipy.stats.poisson.logpmf(X[Δt0:], n[Δt0-1:] * p[Δt0-1:])
loglik.mean()

-221.17259181326324

In [246]:
print('τ',int(τ),'Δt0',int(Δt0))

τ 13 Δt0 2


In [244]:
def r(arr):
    return [str(int(a)) if not np.isinf(a) else '-inf' for a in arr[:-1]]+[str(arr[-1])]
def r_arrays(arr):
    return [r(a) for a in arr]

loglik_to_print = np.array([0]*Δt0+list(loglik))
print('\t'.join(['day', 'loglik', 'X', 'Xsum', 'Y', 'n','p']))
arr = r_arrays(list(zip(range(len(X)),loglik_to_print,X, Xsum, Y, n, p)))
for a in arr:
    print('\t'.join(a))

day	loglik	X	Xsum	Y	n	p
0	0	0	0	0	35	0.1111111111111111
1	0	0	0	35	80	0.1111111111111111
2	-6	1	1	80	132	0.1111111111111111
3	-6	5	6	133	188	0.1111111111111111
4	-10	5	11	194	253	0.1111111111111111
5	-24	1	12	264	331	0.1111111111111111
6	-33	1	13	343	420	0.1111111111111111
7	-42	1	14	433	522	0.1111111111111111
8	-34	9	23	536	629	0.1111111111111111
9	-40	11	34	652	750	0.1111111111111111
10	-29	26	60	784	873	0.1111111111111111
11	-5	76	136	933	966	0.1111111111111111
12	-49	24	160	1102	942	0.16666666666666666
13	-62	42	202	1102	1209	0.16666666666666666
14	-91	45	247	1411	1479	0.16666666666666666
15	-81	78	325	1726	1730	0.16666666666666666
16	-53	136	461	2055	1942	0.16666666666666666
17	-55	158	619	2403	2153	0.16666666666666666
18	-77	155	774	2772	2391	0.16666666666666666
19	-106	149	923	3165	2662	0.16666666666666666
20	-186	108	1031	3585	3001	0.16666666666666666
21	-260	89	1120	4032	3390	0.16666666666666666
22	-406	46	1166	4510	3853	0.16666666666666666
23	-301	134	1300	5019	4264	0.1666666